# Goal:
Join NBA and NCAA data into a table called `player_master_table` for analysis across the NCAA and NBA

Import libraries for dataframe manipulation and database loading

In [1]:
import pandas as pd
import numpy as np
import getpass
import psycopg2
import itertools
from psycopg2.extensions import adapt, register_adapter, AsIs

Connect to our database

In [2]:
mypasswd = getpass.getpass()

conn = psycopg2.connect(database = 'cs20_group4',
                              user = 'jpcp73',
                              host = 'pgsql.dsa.lan',
                              password = mypasswd)

cursor = conn.cursor()

········


In SQL create a temp table for the `player_master_table` to be created from. This will be created from NCAA player data and joined with the NBA player data. In the SQL code is where we normalize the player's names. We remove any punctuation for consistent joining. 

In [3]:
data = pd.read_sql_query("""
DROP TABLE IF EXISTS final_college_season; 

SELECT
    ncaa.player_id
    , ncaa.player_name
    , ncaa.school
    , ncaa.season
    , ncaa.height
    , ncaa.weight
INTO TEMP TABLE final_college_season
FROM (
    SELECT
        pl.player_id
        , TRIM(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(pl.player_name, 'Pöltl', 'Poeltl'), 'Jr.', ''), 'Sr.', ''), '.', ''), '''', ''), '-', ' '), 'Á', 'A'), 'É', 'E'), 'Ó', 'O'), 'Ö', 'O'), 'ß', 'B'), 'á', 'a'), 'ã', 'a'), 'ä', 'a'), 'ç', 'c'), 'è', 'e'), 'é', 'e'), 'ê', 'e'), 'ë', 'e'), 'í', 'i'), 'ï', 'i'), 'ð', 'o'), 'ú', 'u'), 'ü', 'u'), 'ý', 'y'), 'ā', 'a'), 'Ć', 'C'), 'ć', 'c'), 'Č', 'C'), 'č', 'c'), 'đ', 'd'), 'ē', 'e'), 'ğ', 'g'), 'İ', 'I'), 'ı', 'i'), 'ń', 'n'), 'ř', 'r'), 'ş', 's'), 'Š', 'S'), 'š', 's'), 'ū', 'u'), 'Ž', 'Z'), 'ž', 'z'), 'ș', 's'), 'ó', 'o'), 'ô', 'o'), 'ņ', 'n'), 'ö', 'o'), '(', ''), ')', ''), ',', ''), '"', '')) as player_name
        , schools.school
        , pl.season
        , pl.height
        , pl.weight
        , ROW_NUMBER() OVER(PARTITION BY pl.player_id ORDER BY pl.season DESC) as season_count
    FROM ncaa_player_career_stats pl
    LEFT JOIN (
        SELECT
            team.winning_abbr
            , MAX(team.winning_name) as school
        FROM ncaa_advanced_boxscores team
        GROUP BY team.winning_abbr) schools ON UPPER(pl.team_abbreviation) = UPPER(schools.winning_abbr)) ncaa
WHERE ncaa.season_count = 1;

DROP TABLE IF EXISTS first_college_season;

SELECT
    ncaa.player_id
    , ncaa.player_name
    , ncaa.school
    , ncaa.season
INTO TEMP TABLE first_college_season
FROM (
    SELECT
        pl.player_id
        , TRIM(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(pl.player_name, 'Pöltl', 'Poeltl'), 'Jr.', ''), 'Sr.', ''), '.', ''), '''', ''), '-', ' '), 'Á', 'A'), 'É', 'E'), 'Ó', 'O'), 'Ö', 'O'), 'ß', 'B'), 'á', 'a'), 'ã', 'a'), 'ä', 'a'), 'ç', 'c'), 'è', 'e'), 'é', 'e'), 'ê', 'e'), 'ë', 'e'), 'í', 'i'), 'ï', 'i'), 'ð', 'o'), 'ú', 'u'), 'ü', 'u'), 'ý', 'y'), 'ā', 'a'), 'Ć', 'C'), 'ć', 'c'), 'Č', 'C'), 'č', 'c'), 'đ', 'd'), 'ē', 'e'), 'ğ', 'g'), 'İ', 'I'), 'ı', 'i'), 'ń', 'n'), 'ř', 'r'), 'ş', 's'), 'Š', 'S'), 'š', 's'), 'ū', 'u'), 'Ž', 'Z'), 'ž', 'z'), 'ș', 's'), 'ó', 'o'), 'ô', 'o'), 'ņ', 'n'), 'ö', 'o'), '(', ''), ')', ''), ',', ''), '"', '')) as player_name
        , schools.school
        , pl.season
        , ROW_NUMBER() OVER(PARTITION BY pl.player_id ORDER BY pl.season ASC) as season_count
    FROM ncaa_player_career_stats pl
    LEFT JOIN (
        SELECT
            team.winning_abbr
            , MAX(team.winning_name) as school
        FROM ncaa_advanced_boxscores team
        GROUP BY team.winning_abbr) schools ON UPPER(pl.team_abbreviation) = UPPER(schools.winning_abbr)) ncaa
WHERE ncaa.season_count = 1;

DROP TABLE IF EXISTS college;

SELECT
    final.player_id
    , TRIM(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(final.player_name, 'Pöltl', 'Poeltl'), 'Jr.', ''), 'Sr.', ''), '.', ''), '''', ''), '-', ' '), 'Á', 'A'), 'É', 'E'), 'Ó', 'O'), 'Ö', 'O'), 'ß', 'B'), 'á', 'a'), 'ã', 'a'), 'ä', 'a'), 'ç', 'c'), 'è', 'e'), 'é', 'e'), 'ê', 'e'), 'ë', 'e'), 'í', 'i'), 'ï', 'i'), 'ð', 'o'), 'ú', 'u'), 'ü', 'u'), 'ý', 'y'), 'ā', 'a'), 'Ć', 'C'), 'ć', 'c'), 'Č', 'C'), 'č', 'c'), 'đ', 'd'), 'ē', 'e'), 'ğ', 'g'), 'İ', 'I'), 'ı', 'i'), 'ń', 'n'), 'ř', 'r'), 'ş', 's'), 'Š', 'S'), 'š', 's'), 'ū', 'u'), 'Ž', 'Z'), 'ž', 'z'), 'ș', 's'), 'ó', 'o'), 'ô', 'o'), 'ņ', 'n'), 'ö', 'o'), '(', ''), ')', ''), ',', ''), '"', '')) as player_name
    , first.season as first_college_season
    , final.season as final_college_season
    , final.school
    , final.height
    , final.weight
INTO TEMP TABLE college
FROM first_college_season first
JOIN final_college_season final ON UPPER(first.player_id) = UPPER(final.player_id);

DROP TABLE IF EXISTS draft;

SELECT
    TRIM(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(COALESCE(draft.player_name, rank.player), 'Pöltl', 'Poeltl'), 'Jr.', ''), 'Sr.', ''), '.', ''), '''', ''), '-', ' '), 'Á', 'A'), 'É', 'E'), 'Ó', 'O'), 'Ö', 'O'), 'ß', 'B'), 'á', 'a'), 'ã', 'a'), 'ä', 'a'), 'ç', 'c'), 'è', 'e'), 'é', 'e'), 'ê', 'e'), 'ë', 'e'), 'í', 'i'), 'ï', 'i'), 'ð', 'o'), 'ú', 'u'), 'ü', 'u'), 'ý', 'y'), 'ā', 'a'), 'Ć', 'C'), 'ć', 'c'), 'Č', 'C'), 'č', 'c'), 'đ', 'd'), 'ē', 'e'), 'ğ', 'g'), 'İ', 'I'), 'ı', 'i'), 'ń', 'n'), 'ř', 'r'), 'ş', 's'), 'Š', 'S'), 'š', 's'), 'ū', 'u'), 'Ž', 'Z'), 'ž', 'z'), 'ș', 's'), 'ó', 'o'), 'ô', 'o'), 'ņ', 'n'), 'ö', 'o'), '(', ''), ')', ''), ',', ''), '"', ''), '  AnthonyTowns', ' Anthony Towns')) as player_name
    , rank.height
    , rank.weight
    , COALESCE(draft.school, rank.school_or_team) as from
    , COALESCE(draft.year, rank.year) as draft_class
    , CAST(rank.rank as INT) as draft_rank
    , draft.pick as draft_pick
    , draft.team
INTO TEMP TABLE draft
FROM nba_draft_results draft
FULL OUTER JOIN nba_top100_prospect_rankings rank ON UPPER(TRIM(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(draft.player_name, 'Pöltl', 'Poeltl'), 'Jr.', ''), 'Sr.', ''), '.', ''), '''', ''), '-', ' '), 'Á', 'A'), 'É', 'E'), 'Ó', 'O'), 'Ö', 'O'), 'ß', 'B'), 'á', 'a'), 'ã', 'a'), 'ä', 'a'), 'ç', 'c'), 'è', 'e'), 'é', 'e'), 'ê', 'e'), 'ë', 'e'), 'í', 'i'), 'ï', 'i'), 'ð', 'o'), 'ú', 'u'), 'ü', 'u'), 'ý', 'y'), 'ā', 'a'), 'Ć', 'C'), 'ć', 'c'), 'Č', 'C'), 'č', 'c'), 'đ', 'd'), 'ē', 'e'), 'ğ', 'g'), 'İ', 'I'), 'ı', 'i'), 'ń', 'n'), 'ř', 'r'), 'ş', 's'), 'Š', 'S'), 'š', 's'), 'ū', 'u'), 'Ž', 'Z'), 'ž', 'z'), 'ș', 's'), 'ó', 'o'), 'ô', 'o'), 'ņ', 'n'), 'ö', 'o'), '(', ''), ')', ''), ',', ''), '"', ''))) = UPPER(TRIM(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(rank.player, 'Jr.', ''), 'Sr.', ''), '.', ''), '''', ''), '-', ' '), 'Á', 'A'), 'É', 'E'), 'Ó', 'O'), 'Ö', 'O'), 'ß', 'B'), 'á', 'a'), 'ã', 'a'), 'ä', 'a'), 'ç', 'c'), 'è', 'e'), 'é', 'e'), 'ê', 'e'), 'ë', 'e'), 'í', 'i'), 'ï', 'i'), 'ð', 'o'), 'ú', 'u'), 'ü', 'u'), 'ý', 'y'), 'ā', 'a'), 'Ć', 'C'), 'ć', 'c'), 'Č', 'C'), 'č', 'c'), 'đ', 'd'), 'ē', 'e'), 'ğ', 'g'), 'İ', 'I'), 'ı', 'i'), 'ń', 'n'), 'ř', 'r'), 'ş', 's'), 'Š', 'S'), 'š', 's'), 'ū', 'u'), 'Ž', 'Z'), 'ž', 'z'), 'ș', 's'), 'ó', 'o'), 'ô', 'o'), 'ņ', 'n'), 'ö', 'o'), '(', ''), ')', ''), ',', ''), '"', ''), '  AnthonyTowns', ' Anthony Towns')))
                                                    AND CAST(draft.year as INT) = CAST(rank.year as INT);

DROP TABLE IF EXISTS nba;

SELECT
    COALESCE(box.player_id, nba.player_id) as player_id
    , box.player_name
    , box.nba_rookie_season
    , nba.height
    , nba.weight
INTO TEMP TABLE nba
FROM (
    SELECT
        box.player_id
        , TRIM(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(box.player_name, 'Pöltl', 'Poeltl'), 'Jr.', ''), 'Sr.', ''), '.', ''), '''', ''), '-', ' '), 'Á', 'A'), 'É', 'E'), 'Ó', 'O'), 'Ö', 'O'), 'ß', 'B'), 'á', 'a'), 'ã', 'a'), 'ä', 'a'), 'ç', 'c'), 'è', 'e'), 'é', 'e'), 'ê', 'e'), 'ë', 'e'), 'í', 'i'), 'ï', 'i'), 'ð', 'o'), 'ú', 'u'), 'ü', 'u'), 'ý', 'y'), 'ā', 'a'), 'Ć', 'C'), 'ć', 'c'), 'Č', 'C'), 'č', 'c'), 'đ', 'd'), 'ē', 'e'), 'ğ', 'g'), 'İ', 'I'), 'ı', 'i'), 'ń', 'n'), 'ř', 'r'), 'ş', 's'), 'Š', 'S'), 'š', 's'), 'ū', 'u'), 'Ž', 'Z'), 'ž', 'z'), 'ș', 's'), 'ó', 'o'), 'ô', 'o'), 'ņ', 'n'), 'ö', 'o'), '(', ''), ')', ''), ',', ''), '"', '')) as player_name
        , MIN(box.season) as nba_rookie_season
    FROM nba_player_boxscores box
    GROUP BY box.player_id, TRIM(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(box.player_name, 'Pöltl', 'Poeltl'), 'Jr.', ''), 'Sr.', ''), '.', ''), '''', ''), '-', ' '), 'Á', 'A'), 'É', 'E'), 'Ó', 'O'), 'Ö', 'O'), 'ß', 'B'), 'á', 'a'), 'ã', 'a'), 'ä', 'a'), 'ç', 'c'), 'è', 'e'), 'é', 'e'), 'ê', 'e'), 'ë', 'e'), 'í', 'i'), 'ï', 'i'), 'ð', 'o'), 'ú', 'u'), 'ü', 'u'), 'ý', 'y'), 'ā', 'a'), 'Ć', 'C'), 'ć', 'c'), 'Č', 'C'), 'č', 'c'), 'đ', 'd'), 'ē', 'e'), 'ğ', 'g'), 'İ', 'I'), 'ı', 'i'), 'ń', 'n'), 'ř', 'r'), 'ş', 's'), 'Š', 'S'), 'š', 's'), 'ū', 'u'), 'Ž', 'Z'), 'ž', 'z'), 'ș', 's'), 'ó', 'o'), 'ô', 'o'), 'ņ', 'n'), 'ö', 'o'), '(', ''), ')', ''), ',', ''), '"', ''))) box
LEFT JOIN (
    SELECT
        nba.player_id
        , AVG(nba.height) as height
        , AVG(nba.weight) as weight
    FROM nba_player_season_totals nba
    GROUP BY nba.player_id) nba ON box.player_id = nba.player_id;

DROP TABLE IF EXISTS high_school;

SELECT
    TRIM(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(COALESCE(hs.name, t.name), 'Pöltl', 'Poeltl'), 'Jr.', ''), 'Sr.', ''), '.', ''), '''', ''), '-', ' '), 'Á', 'A'), 'É', 'E'), 'Ó', 'O'), 'Ö', 'O'), 'ß', 'B'), 'á', 'a'), 'ã', 'a'), 'ä', 'a'), 'ç', 'c'), 'è', 'e'), 'é', 'e'), 'ê', 'e'), 'ë', 'e'), 'í', 'i'), 'ï', 'i'), 'ð', 'o'), 'ú', 'u'), 'ü', 'u'), 'ý', 'y'), 'ā', 'a'), 'Ć', 'C'), 'ć', 'c'), 'Č', 'C'), 'č', 'c'), 'đ', 'd'), 'ē', 'e'), 'ğ', 'g'), 'İ', 'I'), 'ı', 'i'), 'ń', 'n'), 'ř', 'r'), 'ş', 's'), 'Š', 'S'), 'š', 's'), 'ū', 'u'), 'Ž', 'Z'), 'ž', 'z'), 'ș', 's'), 'ó', 'o'), 'ô', 'o'), 'ņ', 'n'), 'ö', 'o'), '(', ''), ')', ''), ',', ''), '"', '')) as player_name
    , COALESCE(hs.high_school, t.high_school) as high_school
    , COALESCE(hs.position, t.position) as position
    , COALESCE(CAST(hs.height as FLOAT), CAST(t.height as FLOAT)) as height
    , CASE
        WHEN t.weight <> '- '
            THEN CAST(t.weight as FLOAT)
        ELSE hs.weight
      END as weight
    , CAST(t.national_rank as INT) as national_rank
    , CAST(t.position_rank as INT) as position_rank
    , CAST(t.state_rank as INT) as state_rank
    , COALESCE(CAST(hs.recruit_score as FLOAT), CAST(t.recruit_score as FLOAT)) as recruit_score
    , hs.college
    , COALESCE(CAST(hs.class as FLOAT), CAST(t.year as FLOAT)) as class
INTO TEMP TABLE high_school
FROM hs_col_commit_247sports hs
FULL OUTER JOIN hs_top1k_247sports t ON TRIM(UPPER(hs.name)) = TRIM(UPPER(t.name))
                                        AND CAST(hs.class as FLOAT) = CAST(t.year as FLOAT)
                                        AND TRIM(UPPER(hs.high_school)) = TRIM(UPPER(t.high_school));

SELECT
    ROW_NUMBER() OVER (ORDER BY COALESCE(nba.nba_rookie_season, d.draft_class, c.final_college_season), COALESCE(nba.player_name, c.player_name, d.player_name)) as player_id
    , nba.player_id as nba_id
    , c.player_id as ncaa_id
    , COALESCE(nba.player_name, c.player_name, d.player_name, hs.player_name) as player_name
    , COALESCE(c.school, d.from, hs.college) as school_or_team
    , hs.high_school
    , hs.class as high_school_class
    , c.first_college_season
    , c.final_college_season
    , COALESCE(nba.height, d.height, c.height, hs.height) as height
    , COALESCE(nba.weight, d.weight, c.weight, hs.weight) as weight
    , hs.national_rank as high_school_national_rank
    , hs.recruit_score as high_school_recruit_score
    , d.team as drafted_by
    , d.draft_class as nba_draft_year
    , d.draft_rank
    , d.draft_pick as nba_draft_pick
    , nba.nba_rookie_season
    , CASE 
        WHEN nba.player_id IS NOT NULL OR d.draft_pick IS NOT NULL OR d.team IS NOT NULL
            THEN 'NBA'
        WHEN c.player_id IS NOT NULL
            THEN 'NCAA'
        ELSE 'High School'
      END as highest_level
FROM college c
FULL OUTER JOIN draft d ON UPPER(c.player_name) = UPPER(d.player_name)
    AND CAST(c.final_college_season as INT) = CAST(d.draft_class as INT)
FULL OUTER JOIN nba ON UPPER(d.player_name) = UPPER(nba.player_name)
    AND (CAST(nba.nba_rookie_season as INT) = CAST(d.draft_class as INT) + 1
     OR CAST(nba.nba_rookie_season as INT) = CAST(d.draft_class as INT) + 2
     OR CAST(nba.nba_rookie_season as INT) = CAST(d.draft_class as INT) + 3)
FULL OUTER JOIN high_school hs ON UPPER(COALESCE(c.player_name, nba.player_name)) = UPPER(hs.player_name)
                                AND (COALESCE(c.first_college_season, nba.nba_rookie_season) = hs.class + 1
                                    OR COALESCE(c.first_college_season, nba.nba_rookie_season) = hs.class + 2)
WHERE COALESCE(nba.player_name, c.player_name, d.player_name, hs.player_name) IS NOT NULL
""", con = conn)
print(data.shape)
data.head()

(47568, 19)


,player_id,nba_id,ncaa_id,player_name,school_or_team,high_school,high_school_class,first_college_season,final_college_season,height,weight,high_school_national_rank,high_school_recruit_score,drafted_by,nba_draft_year,draft_rank,nba_draft_pick,nba_rookie_season,highest_level
0,1,None,None,Alan Sawyer,UCLA,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WSC,1950.0,NaN,NaN,NaN,NBA
1,2,None,None,Al Henningsen,Northwest Missouri State,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FTW,1950.0,NaN,NaN,NaN,NBA
2,3,None,None,Andy Butchko,Purdue,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MNL,1950.0,NaN,NaN,NaN,NBA
3,4,None,None,Art Burris,Tennessee,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FTW,1950.0,NaN,NaN,NaN,NBA
4,5,None,None,Bernie Adams,Princeton,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PHW,1950.0,NaN,NaN,NaN,NBA


In [5]:
chars = []
for name in data.player_name:
    chars.append([x for x in name])
unique_chars = set(list(itertools.chain.from_iterable(chars)))

In [6]:
c_table = []
for c in data.columns:
    if data[c].dtype in [float, int]:
        c_table.append('{} numeric'.format(c))
    else:
        c_table.append('{} varchar({})'.format(c, max([len(x) for x in data[c] if x is not None])))

In [7]:
create_table = 'DROP TABLE IF EXISTS player_master_table; '
create_table += 'CREATE TABLE player_master_table ({})'.format(', '.join(c for c in c_table))

'DROP TABLE IF EXISTS player_master_table; CREATE TABLE player_master_table (player_id numeric, nba_id varchar(9), ncaa_id varchar(31), player_name varchar(29), school_or_team varchar(38), high_school varchar(56), high_school_class numeric, first_college_season numeric, final_college_season numeric, height numeric, weight numeric, high_school_national_rank numeric, high_school_recruit_score numeric, drafted_by varchar(3), nba_draft_year numeric, draft_rank numeric, nba_draft_pick numeric, nba_rookie_season numeric, highest_level varchar(11))'

In [8]:
cursor.execute(create_table)
conn.commit()

In [9]:
df = data.where(pd.notnull(data), None)

cursor = conn.cursor()

INSERT_SQL = 'INSERT INTO player_master_table'
INSERT_SQL += '({}) VALUES'.format(', '.join([x for x in df.columns]))
INSERT_SQL += '({})'.format(''.join(['%s,' * len(df.columns)])[:-1])

with conn, conn.cursor() as cursor:
    for row in df.itertuples(index=False, name=None):
        cursor.execute(INSERT_SQL, row)

In [10]:
test = pd.read_sql_query("""
SELECT
    *
FROM player_master_table
LIMIT 10
""", con = conn)
test

,player_id,nba_id,ncaa_id,player_name,school_or_team,high_school,high_school_class,first_college_season,final_college_season,height,weight,high_school_national_rank,high_school_recruit_score,drafted_by,nba_draft_year,draft_rank,nba_draft_pick,nba_rookie_season,highest_level
0,1.0,None,None,Alan Sawyer,UCLA,None,None,None,None,None,None,None,None,WSC,1950.0,None,None,None,NBA
1,2.0,None,None,Al Henningsen,Northwest Missouri State,None,None,None,None,None,None,None,None,FTW,1950.0,None,None,None,NBA
2,3.0,None,None,Andy Butchko,Purdue,None,None,None,None,None,None,None,None,MNL,1950.0,None,None,None,NBA
3,4.0,None,None,Art Burris,Tennessee,None,None,None,None,None,None,None,None,FTW,1950.0,None,None,None,NBA
4,5.0,None,None,Bernie Adams,Princeton,None,None,None,None,None,None,None,None,PHW,1950.0,None,None,None,NBA
5,6.0,None,None,Bill Erickson,Illinois,None,None,None,None,None,None,None,None,TRI,1950.0,None,None,None,NBA
6,7.0,None,None,Bill Sharman,Southern California,None,None,None,None,None,None,None,None,WSC,1950.0,None,None,None,NBA
7,8.0,None,None,Billy Adcock,Vanderbilt,None,None,None,None,None,None,None,None,FTW,1950.0,None,None,None,NBA
8,9.0,None,None,Bob Anderson,Loyola (MD),None,None,None,None,None,None,None,None,TRI,1950.0,None,None,None,NBA
9,10.0,None,None,Bob Cope,Montana,None,None,None,None,None,None,None,None,BOS,1950.0,None,None,None,NBA
